In [56]:
!python3 -m pip install seaborn

16053.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 2.2 MB/s eta 0:00:00a 0:00:01


In [54]:
!python3 -m pip install scikit-learn

15865.52s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 6.2 MB/s eta 0:00:00a 0:00:01


In [1]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://localhost:9200")
client.info()
client.ping()

True

In [2]:
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [3]:
print("Got %d Hits:" % resp['hits']['total'])

Got 22329 Hits:


In [57]:
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [34]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5


In [22]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [15]:
def readHtml(html):
    dftable = pd.read_html(html)
    return dftable

In [49]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [25]:
# df_en = pd.DataFrame({'name':name_en,
#                        'sdesc':sdesc_en,
#                     #    'desc':desc_en,
#                        'category':cat_en,
#                        'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [50]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,SparePart,Cooling System,Thermostat,na,na,na
1,zf 1043 010 297 automatic transmission,automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na
2,zf 1060 040 003 automatic transmission,automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na
3,vaico v10 0131 camshaft,camshaft,camshaft,SparePart,Engine,Engine Timing,Camshaft,na,na
4,vaico v26 0302 track control arm,track control arm,track control arm,SparePart,Powertrain & Suspension,Wheel Suspension,Track Control Arm,na,na


In [51]:
df_eng.isna().sum()

name          0
sdesc         0
category      0
catlevel0     0
catlevel1     0
catlevel2     0
catlevel3     6
catlevel4    30
catlevel5    30
dtype: int64

In [52]:
df_eng.shape

(40, 9)

In [58]:
df_eng = df_eng.dropna()

In [59]:
df_eng.shape

(10, 9)

In [60]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,SparePart,Cooling System,Thermostat,na,na,na
1,zf 1043 010 297 automatic transmission,automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na
2,zf 1060 040 003 automatic transmission,automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na
3,vaico v10 0131 camshaft,camshaft,camshaft,SparePart,Engine,Engine Timing,Camshaft,na,na
4,vaico v26 0302 track control arm,track control arm,track control arm,SparePart,Powertrain & Suspension,Wheel Suspension,Track Control Arm,na,na


In [61]:
df_eng.describe()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
count,10,10,10,10,10,10,10,10,10
unique,10,9,9,1,5,6,8,1,1
top,denso dtm82363 thermostat coolant,automatic transmission,automatic transmission,SparePart,Powertrain & Suspension,Automatic Transmission,na,na,na
freq,1,2,2,10,4,3,2,10,10


One hot encoding 

In [62]:
df_eng = pd.get_dummies(df_eng,columns=["category"])
df_eng.head()

,name,sdesc,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5,category_automatic transmission,category_camshaft,category_mounting,category_shock absorber,category_suspension kit,category_tensioner pulley,category_thermostat,category_track control arm,category_wheel hub
0,denso dtm82363 thermostat coolant,thermostat coolant,SparePart,Cooling System,Thermostat,na,na,na,False,False,False,False,False,False,True,False,False
1,zf 1043 010 297 automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na,True,False,False,False,False,False,False,False,False
2,zf 1060 040 003 automatic transmission,automatic transmission,SparePart,Transmission,Automatic Transmission,Automatic Transmission,na,na,True,False,False,False,False,False,False,False,False
3,vaico v10 0131 camshaft,camshaft,SparePart,Engine,Engine Timing,Camshaft,na,na,False,True,False,False,False,False,False,False,False
4,vaico v26 0302 track control arm,track control arm,SparePart,Powertrain & Suspension,Wheel Suspension,Track Control Arm,na,na,False,False,False,False,False,False,False,True,False


In [63]:
df_eng.shape

(10, 17)

In [ ]:
from sklearn.model_selection import train_test_split

